# Processing the data and checking the results

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing


%matplotlib inline
%config IPCompleter.greedy=True

pd.options.mode.chained_assignment = None

In [2]:
order_products_train_df = pd.read_csv("../data/raw/order_products__train.csv")
order_products_prior_df = pd.read_csv("../data/raw/order_products__prior.csv").head(10000)
orders_df = pd.read_csv("../data/raw/orders.csv")
products_df = pd.read_csv("../data/raw/products.csv")
aisles_df = pd.read_csv("../data/raw/aisles.csv")
departments_df = pd.read_csv("../data/raw/departments.csv")
sample_submission_df = pd.read_csv("../data/raw/sample_submission.csv")

## Reduce to most reordered and flatten

In [3]:
value_counts = order_products_prior_df[order_products_prior_df.reordered == 1]["product_id"].value_counts()
value_counts.head()

24852    134
13176    103
47209     59
21903     57
21137     55
Name: product_id, dtype: int64

In [4]:
most_reordered = value_counts.head(200).index
order_prod_p = order_products_prior_df[order_products_prior_df.product_id.isin(most_reordered)]
print("Order_products all:", order_products_prior_df.shape[0])
print("Order_products most reordered", order_prod_p.shape[0])

Order_products all: 10000
Order_products most reordered 3122


In [5]:
order_prod_p.drop(["reordered", "add_to_cart_order"], axis=1)

,order_id,product_id
0,2,33120
1,2,28985
5,2,17794
10,3,24838
12,3,21903
14,3,46667
27,4,25146
30,5,13176
33,5,27966
34,5,23909


In [6]:
binarizer = preprocessing.LabelBinarizer()
binarized_df = pd.DataFrame(binarizer.fit_transform(order_prod_p["product_id"]))
binarized_df.columns = binarizer.classes_
binarized_df.head()

,196,260,432,651,1158,1463,1511,2295,2825,2966,...,47144,47209,47626,47766,47977,48205,48679,49235,49520,49683
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
order_prod_p.drop("product_id", axis=1, inplace=True)
order_prod_p = order_prod_p.join(binarized_df)
order_prod_p.head()

,order_id,add_to_cart_order,reordered,196,260,432,651,1158,1463,1511,...,47144,47209,47626,47766,47977,48205,48679,49235,49520,49683
0,2,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2,6,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,3,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,3,4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
order_prod_p.drop(["add_to_cart_order", "reordered"], axis=1, inplace=True)

In [10]:
df = order_prod_p.groupby("order_id").sum()

In [11]:
df.head()

,196,260,432,651,1158,1463,1511,2295,2825,2966,...,47144,47209,47626,47766,47977,48205,48679,49235,49520,49683
order_id,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
df = pd.merge(df, orders_df, right_on="order_id", left_index=True)

In [13]:
df.head()

,196,260,432,651,1158,1463,1511,2295,2825,2966,...,49235,49520,49683,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
3355525,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2,202279,prior,3,5,9,8.0
3417191,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3,205970,prior,16,5,17,12.0
2958007,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4,178520,prior,36,1,9,7.0
2593373,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5,156122,prior,42,6,16,9.0
2308534,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,9,139016,prior,14,0,19,5.0


## Checking reduced to most reordered and flattened

In [14]:
most_reordered = pd.read_csv("../data/processed/ReducedToMostOrderedFlat.csv")

In [23]:
most_reordered.head()

,user_id,45,196,260,432,1463,1940,2078,2295,3952,...,46969,46979,47144,47209,47626,47672,47766,48679,49235,49683
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [24]:
most_reordered[most_reordered["45"].isnull()]

,user_id,45,196,260,432,1463,1940,2078,2295,3952,...,46969,46979,47144,47209,47626,47672,47766,48679,49235,49683
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
most_reordered.shape[0]

168223